In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1"

In [ ]:
wandb.init(project="tesi", name="bigbird")  # args = TrainingArguments(
wandb.stop()

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, RobertaForSequenceClassification
from datasets import Dataset
tokenizer = AutoTokenizer.from_pretrained(
    'roberta-large'
)
model = RobertaForSequenceClassification.from_pretrained('roberta-large', return_dict=False, num_labels=2
).to('cuda:0')

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    RobertaForSequenceClassification,
)

tokenizer = AutoTokenizer.from_pretrained("ctrl")
model = RobertaForSequenceClassification.from_pretrained(
    "ctrl", return_dict=False, num_labels=2
).to("cuda:0")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
You are using a model of type ctrl to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ctrl and are newly initialized: ['classifier.out_proj.bias', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.6.output.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.4.attention.output.dense.weight', 'encoder.layer.7.attention.self.value.weight', 'encoder.layer.11.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.9.intermediate.dense.weight', 'encoder.layer.10.attention.self.key.weight', 'encoder.layer.11.attention.self.key.bias', 'encoder.lay

In [2]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./nil_el_instanceof.jsonl"
from datasets import Dataset

original_data = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        
        original_data.append(data_line)
print(len(original_data))

7939


In [3]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./aida_train_instanceof.jsonl"
from datasets import Dataset

original_data_aida = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        # additional_question = "<additional> "
        # try:
        #     for related in data_line["most_related"]:
        #         length = len(related)
        #         if len(related) == 4:
        #             additional_question += related[2]
        #             additional_question += " "
        # except:
        #     None
        # additional_question += "</additional>"
        # data_line["context"] += additional_question
        original_data_aida.append(data_line)
print(len(original_data_aida))

18448


In [3]:
import random
#random.shuffle(original_data_aida)
#concat = original_data_aida
#concat =original_data_aida[:int(len(original_data_aida)* 0.4)] + original_data
concat = original_data

random.shuffle(concat)
#print(len(concat))
train_original = concat[: int(len(concat) * 0.95)]
eval_original = concat[int(len(concat) * 0.95) : ]
dataset = Dataset.from_dict(
    {
        "context": [item["question"] for item in train_original],
        "question": [item["context"] for item in train_original],
        "answers": [item["answers"] for item in train_original],
    }
)
dataset_ev = Dataset.from_dict(
    {
        "context": [item["question"] for item in eval_original],
        "question": [item["context"] for item in eval_original],
        "answers": [item["answers"] for item in eval_original],
    }
)

In [4]:
def get_correct_alignement(context, answer):
    """Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here."""
    gold_text = answer["text"][0]
    
    start_idx = context.find(gold_text)
    end_idx = context.find("</ec>", start_idx)
    return start_idx, end_idx
    # if context[start_idx:end_idx] == gold_text:
    #     return start_idx, end_idx       # When the gold label position is good
    # elif context[start_idx-1:end_idx-1] == gold_text:
    #     return start_idx-1, end_idx-1   # When the gold label is off by one character
    # elif context[start_idx-2:end_idx-2] == gold_text:
    #     return start_idx-2, end_idx-2   # When the gold label is off by two character
    # else:
    #     raise ValueError()


# Tokenize our training dataset
def convert_to_features(example):
    try:
        # Tokenize contexts and questions (as pairs of inputs)

        input_pairs = [example["question"], example["context"]]
        encodings = tokenizer.encode_plus(
            input_pairs, pad_to_max_length=True, truncation="only_second"
        )
        try:
            encodings.update(
                {
                   "input_ids": encodings['input_ids'],
                    "attention_mask": encodings["attention_mask"],
                    "labels": 1 if example["answers"]["text"][0] == "Not In Candidates" else 0
                }
            )
            return encodings
        except Exception as e:
            print(e)
       
    except Exception as e:
        print("errors", e)

In [5]:
train_ds = dataset.map(convert_to_features)
eval_ds = dataset_ev.map(convert_to_features) 

Map:   0%|          | 0/7542 [00:00<?, ? examples/s]

/home/rub/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/397 [00:00<?, ? examples/s]

In [ ]:
import joblib
import datasets

In [ ]:
joblib.dump( train_ds, './dumps/aidaTrainIOF.dump',)
joblib.dump( eval_ds, './dumps/aidaEvalIOF.dump',)

In [ ]:
train_ds = joblib.load('./dumps/aidaTrainNERBigBird.dump')
eval_ds = joblib.load('./dumps/aidaEvalNERBigBird.dump')

In [ ]:
wandb.init()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import torch
import wandb
torch.cuda.set_device(0)
torch.cuda.current_device()

#wandb.init(project="tesi", name="robertaAidaLargeNERNIL")  # args = TrainingArguments(
#     f"longformer-finetuned-squad",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

args = TrainingArguments(
    "clsRoberta",
    evaluation_strategy="steps",
    save_strategy="steps",
    report_to="wandb",
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate=3e-5,
    per_device_eval_batch_size=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=20,
    num_train_epochs=5,
    weight_decay=0.02,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

In [9]:
trainer.train()

/home/rub/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/940 [00:00<?, ?it/s]

{'loss': 0.5544, 'learning_rate': 2.6872340425531917e-05, 'epoch': 0.53}


  0%|          | 0/199 [00:00<?, ?it/s]

{'eval_loss': 0.37234142422676086, 'eval_runtime': 5.8804, 'eval_samples_per_second': 67.512, 'eval_steps_per_second': 33.841, 'epoch': 0.53}
{'loss': 0.3792, 'learning_rate': 2.3680851063829785e-05, 'epoch': 1.06}


...

  0%|          | 0/199 [00:00<?, ?it/s]

{'eval_loss': 0.3406025469303131, 'eval_runtime': 5.6134, 'eval_samples_per_second': 70.723, 'eval_steps_per_second': 35.451, 'epoch': 3.18}
{'loss': 0.1348, 'learning_rate': 7.723404255319148e-06, 'epoch': 3.71}


  0%|          | 0/199 [00:00<?, ?it/s]

{'eval_loss': 0.2822650074958801, 'eval_runtime': 5.6163, 'eval_samples_per_second': 70.688, 'eval_steps_per_second': 35.433, 'epoch': 3.71}


KeyboardInterrupt: 

In [39]:
trainer.save_model('./classifierFinal')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dataset[0]

## Prediction and evaluation

In [10]:
def check_brackets(input_string):
    stack = []
    brackets = {"(": ")", "{": "}", "[": "]"}
    for char in input_string:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if not stack or brackets[stack.pop()] != char:
                return False
    return not stack


def process_answer(answer, candidates):
    if answer == "Not In Candidates":
        return answer
    else:
        modified_answer = answer.split("</ec>")[0]
        modified_answer = answer.split(": instance of ")[0]
        modified_answer = answer.split(": instance")[0]
        modified_answer = modified_answer.replace("<s>","")
        modified_answer = modified_answer.replace("</ec","")
        modified_answer = modified_answer.replace("</s>","")
        modified_answer = modified_answer.replace("<s", "")
        modified_answer = modified_answer.replace("</", "")
        modified_answer = modified_answer.replace(" ec", "")
        modified_answer = modified_answer.replace(">","")
        modified_answer = modified_answer.strip()
        if not check_brackets(modified_answer):
            modified_answer = modified_answer.replace("(", "")
            modified_answer = modified_answer.replace(")", "")
        if modified_answer == "Not In Candidates":
            modified_answer = "NIL"
        if modified_answer == "":
            modified_answer = "Not In Candidates"
        if modified_answer not in candidates:
            modified_answer = "Not In Candidates"
        return modified_answer
process_answer("> Alzenau </", ["Alzenau"])

'Alzenau'

In [11]:
import gc
import torch.nn.functional as F
def check_is_nil(input_pairs):
    encodings = tokenizer.encode_plus(
                input_pairs, return_tensors="pt", truncation="only_second"
            ).to("cuda")
    outs = model(
        encodings["input_ids"], attention_mask=encodings["attention_mask"]
    )
    outs = outs[0].cpu().detach().numpy()
    if outs[0][0] < outs[0][1]:
        return True
    else:
        return False
            

def make_prediction(data_entry, nil_prediction):
    with torch.no_grad():
        question = data_entry["input"]
        context = ""
        for item in data_entry["candidates"]:
            context += item + f" </ec> "
        input_pairs = [question, context]
        context += " Not In Candidates </ec> "
        encodings = tokenizer.encode_plus(
            input_pairs, return_tensors="pt", truncation="only_second"
        ).to("cuda")
        outs = model(
            encodings["input_ids"], attention_mask=encodings["attention_mask"]
        )
        answer = ''
        is_nil = check_is_nil(input_pairs)
        if is_nil:
            answer = "Not In Candidates"
        else:
            answer = data_entry['output'][0]['provenance'][0]['title']
        return {
            "correct": data_entry["output"],
            "predicted": answer,
            "candidates": context,
            }

In [ ]:
import wandb
wandb.init()

In [ ]:
from tqdm import tqdm
import json
import torch

results = []
file_path = "./msnbc_iof_nilled.jsonl"
dataset = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)

        dataset.append(data_line)

for item in tqdm(dataset):
    try:
        pred = make_prediction(item, True)
        results.append(pred)
    except Exception as e:
        print(e)

In [15]:
correct

47

In [14]:
correct = 0
correct_trace = []
wrong = []
correct_nil = 0
wrong_nil = 0
full_data = []
for result in tqdm(results):
  processed = result['predicted']
  if processed == result['correct'][0]['answer']:
    correct += 1
    correct_trace.append(result)
    if processed == 'Not In Candidates':
        correct_nil += 1
  
  else:
    if result['correct'][0]['answer'] == 'Not In Candidates':
        wrong_nil += 1
    wrong.append(result)



100%|██████████| 656/656 [00:00<00:00, 1079005.26it/s]


In [16]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.07164634146341463


ZeroDivisionError: division by zero

-  Base: 65.9%
-  Large: 73.21%(enriched) - 74.65% (standard)
-  Large EN: 76.66%
-  large NER: 75.5%
-  Base-NER: 66.35%

## MSNBC large
-  No EN: 91.92%
-  No Instance Of: 92.83% - Yes: 93.44%
-  EN: 92.8
-  Large NER: 92.83%

In [63]:
len(wrong)

65

In [17]:
import pandas as pd
full_df = pd.DataFrame(full_data)
df = pd.DataFrame(wrong)
df_c = pd.DataFrame(correct_trace)

In [22]:
n = 20
offset=2
df_c.iloc[offset *n : offset*n + n]

,correct,predicted,candidates
40,"[{'answer': 'Los Angeles Lakers', 'provenance'...",Los Angeles Lakers,Los Angeles Lakers : instance of basketball te...
41,"[{'answer': 'Miami Heat', 'provenance': [{'tit...",Miami Heat,Heat : instance of process function </ec> Miam...
42,"[{'answer': 'Opportunity (rover)', 'provenance...",Opportunity (rover),Opportunity (rover) : instance of Mars rover <...
43,"[{'answer': 'Saddam Hussein', 'provenance': [{...",Saddam Hussein,Saddam Hussein : instance of human </ec> Halab...
44,"[{'answer': 'Hindu', 'provenance': [{'title': ...",Hindu,Hindu : instance of Unknown </ec> Hindus crick...
45,"[{'answer': 'London Heathrow Airport', 'proven...",London Heathrow Airport,London Heathrow Airport : instance of Unknown ...
46,"[{'answer': 'Grand Rapids, Michigan', 'provena...","Grand Rapids, Michigan","Grand Rapids, Michigan : instance of big city ..."


: 

In [ ]:
df.to_csv("./errorsNil.csv")
df_c.to_csv("./correctNil.csv")